In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [44]:
# Function to Remove Commas and Convert to Floats
def remove_commas(value):
    if isinstance(value, str):
        if ',' in value:
            return float(value.replace(',', ''))

# Read the data from the Central Energy Plant and clean up the data
plant_data = pd.read_csv('Plant Steam Data.csv')
steam_usage = pd.read_csv('Steam Usage Year.csv')
steam_usage = steam_usage.drop(columns=['Unnamed: 2', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19'])
steam_usage = steam_usage.rename(columns={'Unnamed: 0': 'Building', 'Unnamed: 1': 'MeterNo'})
steam_usage = steam_usage.drop([0]).reset_index(drop=True)

In [46]:
# Show the first few rows of the plant data, steam flow is in lb/hr
plant_data.head()

,DATE,TIME,Steam Pressure Leaving Plant,Steam Flow Leaving the Plant,USU.Outside_Air_Temp,USU.Facilities_Steam_Pressure
0,10/16/2024,7:00:00 AM,88.45,23397.28,52.46,69.75
1,10/16/2024,6:00:00 AM,90.40,18584.67,53.20,72.36
2,10/16/2024,5:00:00 AM,90.74,17030.57,54.41,72.54
3,10/16/2024,4:00:00 AM,90.78,15487.72,55.27,72.61
4,10/16/2024,3:00:00 AM,90.79,15050.83,56.00,72.90


In [47]:
# Show the first few rows of the steam usage data, usage data is in lbs/month
steam_usage.head()

,Building,MeterNo,Sep-24,Aug-24,Jul-24,Jun-24,May-24,Apr-24,Mar-24,Feb-24,Jan-24,Dec-23,Nov-23,Oct-23
0,Spectrum,7001,188349.63,175837.97,188216.35,201819.24,254298.24,420523.39,501457.67,670398.40,270591.72,262836.49,374283.56,336207.13
1,HPER,7002,496121.82,428632.85,301262.95,376176.62,551236.28,1241250.35,1754496.48,2184241.26,2533989.86,2360073.89,1954505.15,1633268.00
2,Family Life,7003,0.00,0.00,0.00,0.00,107670.00,196210.00,265330.00,254850.00,343760.00,323570.00,301520.00,194380.00
3,Maeser,7004,245638.00,217663.00,206800.00,222726.00,291209.00,381654.00,550546.00,623689.00,777253.00,730016.00,556323.00,361156.00
4,TSC,7006,198658.72,162076.85,187992.48,246323.48,407983.68,544894.87,673223.07,852632.56,938545.79,873548.39,605475.78,327486.90


In [49]:
print(plant_data.dtypes)
print(steam_usage.dtypes)

DATE                               object
TIME                               object
Steam Pressure Leaving Plant      float64
Steam Flow Leaving the Plant       object
USU.Outside_Air_Temp               object
USU.Facilities_Steam_Pressure      object
dtype: object
Building    object
MeterNo     object
Sep-24      object
Aug-24      object
Jul-24      object
Jun-24      object
May-24      object
Apr-24      object
Mar-24      object
Feb-24      object
Jan-24      object
Dec-23      object
Nov-23      object
Oct-23      object
dtype: object


In [51]:
# Sum all values in the 'Mar-24' column
total_mar_24 = steam_usage['Mar-24'].sum()
print(total_mar_24)

23527183.77


In [52]:
total_flow = plant_data["Steam Flow Leaving the Plant"].sum()
print(total_flow)

TypeError: can only concatenate str (not "int") to str